# Prepocessing

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFECV
from sklearn.linear_model import Lasso
from sklearn.feature_selection import VarianceThreshold

**Bot-Iot**

In [ ]:
data1 = pd.read_csv("/content/Bot-Iot.csv_0_row_100000.csv")
data2 = pd.read_csv("/content/Bot-Iot.csv_9_row_100000.csv")
data3 = pd.read_csv("/content/Bot-Iot.csv_1_row_100000.csv")
data4 = pd.read_csv("/content/Bot-Iot.csv_2_row_100000.csv")
data5 = pd.read_csv("/content/Bot-Iot.csv_3_row_100000.csv")
data6 = pd.read_csv("/content/Bot-Iot.csv_4_row_100000.csv")
data7 = pd.read_csv("/content/Bot-Iot.csv_5_row_100000.csv")
data8 = pd.read_csv("/content/Bot-Iot.csv_6_row_100000.csv")
data9 = pd.read_csv("/content/Bot-Iot.csv_7_row_100000.csv")
data10 = pd.read_csv("/content/Bot-Iot.csv_8_row_100000.csv")

data = pd.concat([data1, data2, data3, data4, data5, data6, data7, data8, data9, data10])

In [ ]:
data = data.sample(frac=0.2, random_state=10)

In [ ]:
data.shape

(200000, 48)

In [ ]:
data.columns

Index(['Unnamed: 0', 'row_id', 'pkSeqID', 'stime', 'flgs', 'flgs_number',
       'proto', 'proto_number', 'saddr', 'sport', 'daddr', 'dport', 'pkts',
       'bytes', 'state', 'state_number', 'ltime', 'seq', 'dur', 'mean',
       'stddev', 'sum', 'min', 'max', 'spkts', 'dpkts', 'sbytes', 'dbytes',
       'rate', 'srate', 'drate', 'TnBPSrcIP', 'TnBPDstIP', 'TnP_PSrcIP',
       'TnP_PDstIP', 'TnP_PerProto', 'TnP_Per_Dport', 'AR_P_Proto_P_SrcIP',
       'AR_P_Proto_P_DstIP', 'N_IN_Conn_P_DstIP', 'N_IN_Conn_P_SrcIP',
       'AR_P_Proto_P_Sport', 'AR_P_Proto_P_Dport',
       'Pkts_P_State_P_Protocol_P_DestIP', 'Pkts_P_State_P_Protocol_P_SrcIP',
       'attack', 'category', 'subcategory'],
      dtype='object')

In [ ]:
data['category'].unique()

array(['DDoS', 'Reconnaissance', 'DoS', 'Theft', 'Normal'], dtype=object)

In [ ]:
label_encoder = LabelEncoder()
data = data.dropna()
data['category'] = label_encoder.fit_transform(data['category'])
data['subcategory'] = label_encoder.fit_transform(data['subcategory'])

In [ ]:
data = data.select_dtypes(include='number')

In [ ]:
clean_data = data
clean_data=clean_data.loc[:,(clean_data!=0).any(axis=0)]
X,y= clean_data.drop(['category'],axis=1),clean_data['category']
clean_data_nomialzion=X/X.mean()
vt=VarianceThreshold(threshold=0.003) # Dùng model VT để đánh dấu các cột có gì gị threshold bé hơn 0.003
_=vt.fit(clean_data_nomialzion)
mask = vt.get_support() # Đánh dấu các cột có giá trị bé hơn 0.003 bằng mask
X_reduced=X.loc[:, mask] # Xóa các cột

def identify_correlated(df, threshold):
    """
    A function to identify highly correlated features.
    """
    # Tính ma trận tương quan
    matrix = df.corr().abs()

    mask = np.triu(np.ones_like(matrix, dtype=bool))

    # Tạo ra ma trận
    reduced_matrix = matrix.mask(mask)

    # Tìm các cột có threshold lơn hơn
    to_drop = [c for c in reduced_matrix.columns if any(reduced_matrix[c] > threshold)]

    return to_drop
to_drop = identify_correlated(X_reduced, threshold=0.9)
len(to_drop)
X_reduced.drop(to_drop,axis=1,inplace= True)
X_train,X_test,Y_train,Y_test=train_test_split(X_reduced,y,test_size=0.3,random_state=1)
rfecv = RFECV( # Init rfecv
    estimator=LinearRegression(),
    scoring="r2",
    cv=3,
    n_jobs=-1,
    min_features_to_select=5,
    step=5,
)

_ = rfecv.fit(X_test,Y_test) # Fit

mask = rfecv.support_ # Get a boolean mask for the features to keep
X_final = X_reduced.loc[:, mask].copy()

X_final.shape

<ipython-input-8-7200376dd7ee>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_reduced.drop(to_drop,axis=1,inplace= True)


(200000, 5)

In [ ]:
X_final.columns

Index(['proto_number', 'stddev', 'min', 'max', 'subcategory'], dtype='object')

In [ ]:
X_final = X_final.drop(['subcategory'],axis=1)

In [ ]:
arr_col = list(X_final.columns)
arr_col.append('category')
data = data[arr_col]

**Iot**

In [ ]:
data1 = pd.read_csv("/content/Iot.csv_0_row_100000.csv")
data2 = pd.read_csv("/content/Iot.csv_1_row_100000.csv")
data3 = pd.read_csv("/content/Iot.csv_2_row_100000.csv")
data4 = pd.read_csv("/content/Iot.csv_3_row_100000.csv")
data5 = pd.read_csv("/content/Iot.csv_4_row_100000.csv")
data6 = pd.read_csv("/content/Iot.csv_5_row_100000.csv")
data7 = pd.read_csv("/content/Iot.csv_6_row_100000.csv")
data = pd.concat([data1, data2, data3, data4, data5, data6, data7])

In [ ]:
data.shape

(625783, 86)

In [ ]:
data['Target'].unique()

array(['MITM ARP Spoofing', 'Normal', 'DoS', 'Scan', 'Mirai'],
      dtype=object)

In [ ]:
label_encoder = LabelEncoder()
data = data.dropna()
data['Target'] = label_encoder.fit_transform(data['Target'])
data = data.select_dtypes(include='number')

In [ ]:
data['Target'].unique()

array([1, 3, 0, 4, 2])

In [ ]:
data['Target'].value_counts()

2    415677
4     75265
0     59391
3     40073
1     35377
Name: Target, dtype: int64

In [ ]:
clean_data = data
clean_data=clean_data.loc[:,(clean_data!=0).any(axis=0)]
X,y= clean_data.drop(['Target'],axis=1),clean_data['Target']
clean_data_nomialzion=X/X.mean()
vt=VarianceThreshold(threshold=0.003) # Dùng model VT để đánh dấu các cột có gì gị threshold bé hơn 0.003
_=vt.fit(clean_data_nomialzion)
mask = vt.get_support() # Đánh dấu các cột có giá trị bé hơn 0.003 bằng mask
X_reduced=X.loc[:, mask] # Xóa các cột

def identify_correlated(df, threshold):
    """
    A function to identify highly correlated features.
    """
    # Tính ma trận tương quan
    matrix = df.corr().abs()

    mask = np.triu(np.ones_like(matrix, dtype=bool))

    # Tạo ra ma trận
    reduced_matrix = matrix.mask(mask)

    # Tìm các cột có threshold lơn hơn
    to_drop = [c for c in reduced_matrix.columns if any(reduced_matrix[c] > threshold)]

    return to_drop
to_drop = identify_correlated(X_reduced, threshold=0.8)
len(to_drop)
X_reduced.drop(to_drop,axis=1,inplace= True)
X_train,X_test,Y_train,Y_test=train_test_split(X_reduced,y,test_size=0.3,random_state=1)
rfecv = RFECV( # Init rfecv
    estimator=LinearRegression(),
    scoring="r2",
    cv=3,
    n_jobs=-1,
    min_features_to_select=5,
    step=5,
)

_ = rfecv.fit(X_test,Y_test) # Fit

mask = rfecv.support_ # Get a boolean mask for the features to keep
X_final = X_reduced.loc[:, mask].copy()

X_final.shape

<ipython-input-13-6abdfd450e7e>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_reduced.drop(to_drop,axis=1,inplace= True)


(625783, 35)

In [ ]:
X_final.columns

Index(['row_id', 'Src_Port', 'Dst_Port', 'Protocol', 'Fwd_Pkt_Len_Std',
       'Bwd_Pkt_Len_Std', 'Fwd_IAT_Max', 'Fwd_IAT_Min', 'Bwd_IAT_Std',
       'Bwd_IAT_Min', 'Fwd_Pkts/s', 'Bwd_Pkts/s', 'Pkt_Len_Var',
       'FIN_Flag_Cnt', 'SYN_Flag_Cnt', 'RST_Flag_Cnt', 'PSH_Flag_Cnt',
       'ACK_Flag_Cnt', 'URG_Flag_Cnt', 'CWE_Flag_Count', 'ECE_Flag_Cnt',
       'Down/Up_Ratio', 'Fwd_Seg_Size_Avg', 'Bwd_Seg_Size_Avg',
       'Subflow_Fwd_Byts', 'Subflow_Bwd_Pkts', 'Subflow_Bwd_Byts',
       'Init_Bwd_Win_Byts', 'Fwd_Act_Data_Pkts', 'Active_Std', 'Active_Max',
       'Active_Min', 'Idle_Std', 'Idle_Max', 'Idle_Min'],
      dtype='object')

In [ ]:
X_final = X_final.drop(['row_id', 'Src_Port', 'Dst_Port', 'Protocol'],axis=1)

In [ ]:
arr_col = list(X_final.columns)
arr_col.append('Target')
data = data[arr_col]

**CIC-IDS-2017**

In [ ]:
data1 = pd.read_csv("/content/CIC-IDS-2017.csv_0_row_100000.csv")
data2 = pd.read_csv("/content/CIC-IDS-2017.csv_1_row_100000.csv")
data3 = pd.read_csv("/content/CIC-IDS-2017.csv_2_row_100000.csv")
data4 = pd.read_csv("/content/CIC-IDS-2017.csv_3_row_100000.csv")
data5 = pd.read_csv("/content/CIC-IDS-2017.csv_4_row_100000.csv")
data6 = pd.read_csv("/content/CIC-IDS-2017.csv_5_row_100000.csv")
data7 = pd.read_csv("/content/CIC-IDS-2017.csv_6_row_100000.csv")
data8 = pd.read_csv("/content/CIC-IDS-2017.csv_7_row_100000.csv")
data9 = pd.read_csv("/content/CIC-IDS-2017.csv_8_row_100000.csv")
data10 = pd.read_csv("/content/CIC-IDS-2017.csv_9_row_100000.csv")

data = pd.concat([data1,data2,data3,data4,data5,data6,data7,data8,data9,data10])

In [ ]:
data = data.sample(frac=0.1,random_state=100)

In [ ]:
data.shape

(100000, 87)

In [ ]:
data['Target'].unique()

array(['BENIGN', 'DoS Hulk', 'DDoS', 'PortScan', 'DoS GoldenEye',
       'DoS slowloris', 'FTP-Patator', 'DoS Slowhttptest',
       'Web Attack - Brute Force', 'SSH-Patator', 'Bot',
       'Web Attack - XSS', 'Infiltration'], dtype=object)

In [ ]:
label_encoder = LabelEncoder()
data = data.dropna()
data['Target'] = label_encoder.fit_transform(data['Target'])
data = data.select_dtypes(include='number')

<ipython-input-6-bff177864a25>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Target'] = label_encoder.fit_transform(data['Target'])


In [ ]:
data['Target'].unique()

array([ 0,  4,  2,  9,  3,  6,  7,  5, 11, 10,  1, 12,  8])

In [ ]:
clean_data = data
clean_data=clean_data.loc[:,(clean_data!=0).any(axis=0)]
X,y= clean_data.drop(['Target'],axis=1),clean_data['Target']
clean_data_nomialzion=X/X.mean()
vt=VarianceThreshold(threshold=0.003) # Dùng model VT để đánh dấu các cột có gì gị threshold bé hơn 0.003
_=vt.fit(clean_data_nomialzion)
mask = vt.get_support() # Đánh dấu các cột có giá trị bé hơn 0.003 bằng mask
X_reduced=X.loc[:, mask] # Xóa các cột

def identify_correlated(df, threshold):
    """
    A function to identify highly correlated features.
    """
    # Tính ma trận tương quan
    matrix = df.corr().abs()

    mask = np.triu(np.ones_like(matrix, dtype=bool))

    # Tạo ra ma trận
    reduced_matrix = matrix.mask(mask)

    # Tìm các cột có threshold lơn hơn
    to_drop = [c for c in reduced_matrix.columns if any(reduced_matrix[c] > threshold)]

    return to_drop
to_drop = identify_correlated(X_reduced, threshold=0.8)
len(to_drop)
X_reduced.drop(to_drop,axis=1,inplace= True)
X_train,X_test,Y_train,Y_test=train_test_split(X_reduced,y,test_size=0.3,random_state=1)
rfecv = RFECV( # Init rfecv
    estimator=LinearRegression(),
    scoring="r2",
    cv=3,
    n_jobs=-1,
    min_features_to_select=5,
    step=5,
)

_ = rfecv.fit(X_test,Y_test) # Fit

mask = rfecv.support_ # Get a boolean mask for the features to keep
X_final = X_reduced.loc[:, mask].copy()

X_final.shape

<ipython-input-8-6abdfd450e7e>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_reduced.drop(to_drop,axis=1,inplace= True)


(99949, 32)

In [ ]:
X_final.columns

Index(['row_id', 'Source Port', 'Destination Port', 'Fwd Packet Length Min',
       'Bwd Packet Length Min', 'Flow IAT Min', 'Bwd IAT Total', 'Bwd IAT Max',
       'Bwd IAT Min', 'Fwd Packets/s', 'Bwd Packets/s', 'Min Packet Length',
       'FIN Flag Count', 'SYN Flag Count', 'PSH Flag Count', 'ACK Flag Count',
       'URG Flag Count', 'CWE Flag Count', 'ECE Flag Count', 'Down/Up Ratio',
       'Avg Fwd Segment Size', 'Avg Bwd Segment Size', 'Subflow Fwd Bytes',
       'Init_Win_bytes_forward', 'Init_Win_bytes_backward', 'act_data_pkt_fwd',
       'min_seg_size_forward', 'Active Std', 'Active Max', 'Active Min',
       'Idle Std', 'Idle Min'],
      dtype='object')

In [ ]:
X_final = X_final.drop(['row_id', 'Source Port', 'Destination Port'],axis=1)

In [ ]:
arr_col = list(X_final.columns)
arr_col.append('Target')
data = data[arr_col]

# Chia dataset và gộp lại thành các subset

In [ ]:
data.shape

(200000, 5)

In [ ]:
data.columns

Index(['proto_number', 'stddev', 'min', 'max', 'category'], dtype='object')

In [ ]:
data['category'].value_counts()

0    105002
1     89875
3      5083
2        37
4         3
Name: category, dtype: int64

In [ ]:
arr = list(data['category'].unique())
print(arr)

[0, 3, 1, 4, 2]


In [ ]:
dict_dataset = {}
for i in arr:
  temp = data[data['category'] == i]
  dict_dataset[i] = temp

In [ ]:
new_data = []
for i in range(len(arr)):
  temp = []
  for j in range(len(arr)):
    if j != i :
      tmp = dict_dataset[j]
      temp.append(tmp)

  dt = pd.concat(temp)

  new_data.append(dt)

In [ ]:
x = data.shape[0] // len(arr)
testset = []
for i in range(len(arr)):
  tmp = data.iloc[i*x:(i+1)*x]
  testset.append(tmp)

In [ ]:
!pip install -q flwr[simulation] torch torchvision matplotlib

In [ ]:
from collections import OrderedDict
from typing import Dict, List, Optional, Tuple

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split

import flwr as fl
from flwr.common import Metrics

#Chọn loại thiết bị để huấn luyện
DEVICE = torch.device("cpu")  # Try "cuda" to train on GPU
print(
    f"Training on {DEVICE} using PyTorch {torch.__version__} and Flower {fl.__version__}"
)

Training on cpu using PyTorch 2.1.0+cu118 and Flower 1.6.0


In [ ]:
import torch
from torch.utils.data import Dataset,TensorDataset

In [ ]:
def transform_to_tensor(X,y):
  X_train = X.to_numpy()
  X_train = torch.tensor(X_train)
  y_train = y.to_numpy()
  y_train = torch.tensor(y_train)
  return X_train, y_train


class CustomTensorDataset(Dataset):
    """TensorDataset with support of transforms.
    """
    def __init__(self, tensors, transform=None):
        assert all(tensors[0].size(0) == tensor.size(0) for tensor in tensors)
        self.tensors = tensors
        self.transform = transform

    def __getitem__(self, index):
        x = self.tensors[0][index]

        if self.transform:
            x = self.transform(x)

        y = self.tensors[1][index]

        return x, y

    def __len__(self):
        return self.tensors[0].size(0)

In [ ]:
BATCH_SIZE = 64

def partition_dataset(data):
  trainloaders = []
  valloaders = []
  testloaders = []
  test_X = []
  test_y = []
  for dt in data:
    X = dt.drop('category',axis=1)
    y = dt['category']
    X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=0.2,random_state=10)
    X_val, y_val = transform_to_tensor(X_val, y_val)
    X_train, y_train = transform_to_tensor(X_train, y_train)
    trainset = CustomTensorDataset(tensors=(X_train, y_train), transform=None)
    valset = CustomTensorDataset(tensors=(X_val,y_val),transform=None)
    trainloaders.append(DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True))
    valloaders.append(DataLoader(valset, batch_size=BATCH_SIZE))


  return trainloaders, valloaders

In [ ]:
trainloaders, valloaders = partition_dataset(new_data)

In [ ]:
param = 0
trainloader = trainloaders[0]
for feature,label in trainloader:
  for j in feature:
    param = len(j)
    break
  break
print(param)

4


In [ ]:
class Net(nn.Module): # nn.Module là một base class trong PyTorch để xây dựng các mô hình neural network
    def __init__(self, x) -> None:
        super(Net, self).__init__()
        self.fc1 = nn.Linear(x, 100)
        self.fc2 = nn.Linear(100, 100)
        self.fc3 = nn.Linear(100, 100)
        self.fc4 = nn.Linear(100,len(arr))

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = F.relu(self.fc1(x.float()))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

# Federated Learning

In [ ]:
def train(net, trainloader,epochs: int,verbose=False):
  criterion = torch.nn.CrossEntropyLoss() # Hàm mất mát để đánh giá sai số
  optimizer = torch.optim.Adam(net.parameters()) # Dùng để điều chỉnh các tham số
  net.train()
  for epoch in range(epochs):
    correct, total, epoch_loss = 0, 0, 0.0
    for features, label in trainloader:
      features, label = features.to(DEVICE), label.to(DEVICE)
      optimizer.zero_grad()
      outputs = net(features)
      loss = criterion(outputs, label)
      loss.backward()
      optimizer.step()
      epoch_loss += loss
      total += label.size(0)
      correct += (torch.max(outputs.data, 1)[1] == label).sum().item()
    epoch_loss /= len(trainloader.dataset)
    epoch_acc = correct / total
    if verbose: # In ra kết quả nếu verbose = true
            print(f"Epoch {epoch+1}: train loss {epoch_loss}, accuracy {epoch_acc}")
def test(net, testloader):
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    dict_eval = {}
    dict_all_eval = {}
    for i in range(len(arr)):
      dict_eval[i] = {'TP':0,'FP':0,'FN':0}
      dict_all_eval[i] = {'precision': 0,'recall': 0,'f1-score': 0}
    with torch.no_grad(): # Các phép tính trong vòng lặp không được ghi lại để tiết kiệm bộ nhớ
        for features, label in testloader:
            features, label = features.to(DEVICE), label.to(DEVICE)
            outputs = net(features)
            loss += criterion(outputs, label).item()
            _, predicted = torch.max(outputs.data, 1)
            total += label.size(0)
            correct += (predicted == label).sum().item()
            for i in dict_eval:
              for j in range(len(label)):
                if i == label[j]:
                  if predicted[j] == label[j]:
                    dict_eval[i]['TP'] += 1
                  else:
                    dict_eval[i]['FP'] += 1
                else:
                  if predicted[j] != label[j]:
                    dict_eval[i]['FN'] += 1
    for i in dict_eval:
      t1 = 0
      t2 = 0
      if dict_eval[i]['TP'] + dict_eval[i]['FP'] != 0:
        t1 = dict_eval[i]['TP'] / (dict_eval[i]['TP'] + dict_eval[i]['FP'])
        dict_all_eval[i]['precision'] = t1
      if dict_eval[i]['TP'] + dict_eval[i]['FN'] != 0:
        t2 = dict_eval[i]['TP'] / (dict_eval[i]['TP'] + dict_eval[i]['FN'])
        dict_all_eval[i]['recall'] = t2
      if t1 + t2 != 0:
        dict_all_eval[i]['f1-score'] = 2*t1*t2/(t1+t2)

    print(dict_all_eval)
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy

In [ ]:
# Lấy tham số của mạng ở sau khi client chạy
def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()] # Lấy tất cả các tham số của mạng sau đó chuyển chúng về dạng numpy

# Cập nhật tham số của mạng lên server
def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters) # Tạo một từ điển
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)

In [ ]:
class FlowerClient(fl.client.NumPyClient):#Client
    def __init__(self, net, trainloader, valloader):
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):#lấy tham số sau khi tổng hợp từ server
        return get_parameters(self.net)

    def fit(self, parameters, config):#điều chỉnh tham số để thực hiện huấn luyên
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=1)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):#đánh giá loss,accuracy
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        print(f"Client-side evaluation loss {loss} / accuracy {accuracy}")
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}

In [ ]:
def client_fn(cid: str) -> FlowerClient:#hàm mô phỏng 1 máy client
    """Create a Flower client representing a single organization."""

    # Load model
    net = Net(param).to(DEVICE)

    # Note: each client gets a different trainloader/valloader, so each client
    # will train and evaluate on their own unique data
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]

    # Create a  single Flower client representing a single organization
    return FlowerClient(net, trainloader, valloader)

In [ ]:
def fit_config(server_round: int):
    """Return training configuration dict for each round.

    Perform two rounds of training with one local epoch, increase to two local
    epochs afterwards.
    """
    config = {
        "server_round": server_round,  # The current round of federated learning
        "local_epochs": 2,
    }
    return config

In [ ]:
#hàm đánh giá ở server
def evaluate(
    server_round: int,
    parameters: fl.common.NDArrays,
    config: Dict[str, fl.common.Scalar],
) -> Optional[Tuple[float, Dict[str, fl.common.Scalar]]]:
    net = Net(param).to(DEVICE)

    set_parameters(net, parameters)  # Update model with the latest parameters


In [ ]:
params = get_parameters(Net(param))

# Create FedAvg strategy
strategy = fl.server.strategy.FedAvg( # Có thể tùy chọn chiến lược khác như FedAdagrad
    fraction_fit=1.0,  # Tỷ lệ các thiết bị người dùng được chọn để huấn luyện
    fraction_evaluate=0.5,  # Tỷ lệ các thiết bị người dùng được chọn để đánh giá hiệu suất
    min_fit_clients=len(arr)-1,  # Số tối thiểu các thiết bị người dùng cần có trong quá trình huấn luyện
    min_evaluate_clients=5,  # Số tối thiểu các thiết bị người dùng cần có trong quá trình đánh giá
    min_available_clients=len(arr),  # Wait until all 10 clients are available
    initial_parameters=fl.common.ndarrays_to_parameters(params),
    evaluate_fn=evaluate,  # Pass the evaluation function
    on_fit_config_fn=fit_config,  # Pass the fit_config function
)


# Specify client resources if you need GPU (defaults to 1 CPU and 0 GPU)
client_resources = None # Xác định các tài nguyên của máy khách



# Start simulation
fl.simulation.start_simulation( # Mô phỏng bằng cách gọi hàm
    client_fn=client_fn, # Một hàm để tạo ra các máy khách
    num_clients=len(arr), # Số lượng máy khách
    config=fl.server.ServerConfig(num_rounds=5), # Cấu hình cho máy chủ
    strategy=strategy, # Chiến lược huấn luyện và tổ chức dữ liệu
    client_resources=client_resources, # Tài nguyên của máy khách (số lượng GPU)
)

INFO flwr 2023-12-09 16:16:57,741 | app.py:178 | Starting Flower simulation, config: ServerConfig(num_rounds=5, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=5, round_timeout=None)
2023-12-09 16:16:59,797	INFO worker.py:1621 -- Started a local Ray instance.
INFO flwr 2023-12-09 16:17:02,666 | app.py:213 | Flower VCE: Ray initialized with resources: {'memory': 7866185319.0, 'object_store_memory': 3933092659.0, 'node:172.28.0.12': 1.0, 'node:__internal_head__': 1.0, 'CPU': 2.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'memory': 7866185319.0, 'object_store_memory': 3933092659.0, 'node:172.28.0.12': 1.0, 'node:__internal_head__': 1.0, 'CPU': 2.0}
INFO flwr 2023-12-09 16:17:02,671 | app.py:219 | Optimize your simulation with Flower VCE: https://flower.dev/docs/framework/how-to-run-simulations.html
INFO:flwr:Optimize your simulation with Flower VCE: https://flower.dev/docs/framework/how-to-run-simulations.html
INFO flwr 2023-12-09 16:

(DefaultActor pid=21642) {0: {'precision': 0, 'recall': 0.0, 'f1-score': 0}, 1: {'precision': 0.5309330063538067, 'recall': 0.9000377928949358, 'f1-score': 0.667881932272313}, 2: {'precision': 0.0, 'recall': 0.0, 'f1-score': 0}, 3: {'precision': 0.0, 'recall': 0.0, 'f1-score': 0}, 4: {'precision': 0.0, 'recall': 0.0, 'f1-score': 0}}
(DefaultActor pid=21642) Client-side evaluation loss 0.01575748658807654 / accuracy 0.5013684210526316
(DefaultActor pid=21642) {0: {'precision': 0.8810991636798089, 'recall': 0.6573374215630348, 'f1-score': 0.7529455005819533}, 1: {'precision': 0.5252525252525253, 'recall': 0.7274963486816819, 'f1-score': 0.6100493118896445}, 2: {'precision': 0.0, 'recall': 0.0, 'f1-score': 0}, 3: {'precision': 0.0, 'recall': 0.0, 'f1-score': 0}, 4: {'precision': 0, 'recall': 0.0, 'f1-score': 0}} [repeated 2x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-loggi

DEBUG flwr 2023-12-09 16:18:03,974 | server.py:187 | evaluate_round 1 received 5 results and 0 failures
DEBUG:flwr:evaluate_round 1 received 5 results and 0 failures
WARNING flwr 2023-12-09 16:18:03,977 | fedavg.py:273 | No evaluate_metrics_aggregation_fn provided
DEBUG flwr 2023-12-09 16:18:03,979 | server.py:222 | fit_round 2: strategy sampled 5 clients (out of 5)
DEBUG:flwr:fit_round 2: strategy sampled 5 clients (out of 5)
DEBUG flwr 2023-12-09 16:18:35,451 | server.py:236 | fit_round 2 received 5 results and 0 failures
DEBUG:flwr:fit_round 2 received 5 results and 0 failures
DEBUG flwr 2023-12-09 16:18:35,471 | server.py:173 | evaluate_round 2: strategy sampled 5 clients (out of 5)
DEBUG:flwr:evaluate_round 2: strategy sampled 5 clients (out of 5)


(DefaultActor pid=21642) {0: {'precision': 0.7985252140818269, 'recall': 0.8047657860670279, 'f1-score': 0.8016333548248442}, 1: {'precision': 0.7736258840563569, 'recall': 0.7660747766626227, 'f1-score': 0.7698318140256575}, 2: {'precision': 0.0, 'recall': 0.0, 'f1-score': 0}, 3: {'precision': 0, 'recall': 0.0, 'f1-score': 0}, 4: {'precision': 0.0, 'recall': 0.0, 'f1-score': 0}} [repeated 3x across cluster]
(DefaultActor pid=21642) Client-side evaluation loss 0.006464264505628245 / accuracy 0.786912579519803 [repeated 3x across cluster]
(DefaultActor pid=21643) {0: {'precision': 0, 'recall': 0.0, 'f1-score': 0}, 1: {'precision': 0.7707613421023297, 'recall': 0.9465434633812457, 'f1-score': 0.849655935119194}, 2: {'precision': 0.0, 'recall': 0.0, 'f1-score': 0}, 3: {'precision': 0.26507177033492824, 'recall': 0.06291165114694526, 'f1-score': 0.10168869309838473}, 4: {'precision': 0.0, 'recall': 0.0, 'f1-score': 0}} [repeated 3x across cluster]
(DefaultActor pid=21643) Client-side evalu

DEBUG flwr 2023-12-09 16:18:53,020 | server.py:187 | evaluate_round 2 received 5 results and 0 failures
DEBUG:flwr:evaluate_round 2 received 5 results and 0 failures
DEBUG flwr 2023-12-09 16:18:53,026 | server.py:222 | fit_round 3: strategy sampled 5 clients (out of 5)
DEBUG:flwr:fit_round 3: strategy sampled 5 clients (out of 5)
DEBUG flwr 2023-12-09 16:19:22,870 | server.py:236 | fit_round 3 received 5 results and 0 failures
DEBUG:flwr:fit_round 3 received 5 results and 0 failures
DEBUG flwr 2023-12-09 16:19:22,890 | server.py:173 | evaluate_round 3: strategy sampled 5 clients (out of 5)
DEBUG:flwr:evaluate_round 3: strategy sampled 5 clients (out of 5)


...

(DefaultActor pid=21642) {0: {'precision': 0, 'recall': 0.0, 'f1-score': 0}, 1: {'precision': 0.7979043584884629, 'recall': 0.9482678677883023, 'f1-score': 0.8666121855988378}, 2: {'precision': 0.0, 'recall': 0.0, 'f1-score': 0}, 3: {'precision': 0.26507177033492824, 'recall': 0.07073544433094994, 'f1-score': 0.11167103406571255}, 4: {'precision': 0.0, 'recall': 0.0, 'f1-score': 0}} [repeated 2x across cluster]
(DefaultActor pid=21642) Client-side evaluation loss 0.00605011891063891 / accuracy 0.7680526315789473 [repeated 2x across cluster]
(DefaultActor pid=21643) {0: {'precision': 0.8924807747080604, 'recall': 0.8108073141279972, 'f1-score': 0.8496859040990645}, 1: {'precision': 0.7978117673328123, 'recall': 0.8250779355732595, 'f1-score': 0.8112158020206607}, 2: {'precision': 0, 'recall': 0.0, 'f1-score': 0}, 3: {'precision': 0.24530168150346193, 'recall': 0.04041062408342839, 'f1-score': 0.06939003917179631}, 4: {'precision': 0.0, 'recall': 0.0, 'f1-score': 0}}
(DefaultActor pid=21

DEBUG flwr 2023-12-09 16:20:27,010 | server.py:187 | evaluate_round 4 received 5 results and 0 failures
DEBUG:flwr:evaluate_round 4 received 5 results and 0 failures
DEBUG flwr 2023-12-09 16:20:27,016 | server.py:222 | fit_round 5: strategy sampled 5 clients (out of 5)
DEBUG:flwr:fit_round 5: strategy sampled 5 clients (out of 5)
DEBUG flwr 2023-12-09 16:20:57,582 | server.py:236 | fit_round 5 received 5 results and 0 failures
DEBUG:flwr:fit_round 5 received 5 results and 0 failures
DEBUG flwr 2023-12-09 16:20:57,600 | server.py:173 | evaluate_round 5: strategy sampled 5 clients (out of 5)
DEBUG:flwr:evaluate_round 5: strategy sampled 5 clients (out of 5)


(DefaultActor pid=21643) {0: {'precision': 0.8761655566127498, 'recall': 0.961020663744521, 'f1-score': 0.9166334859645631}, 1: {'precision': 0, 'recall': 0.0, 'f1-score': 0}, 2: {'precision': 0.0, 'recall': 0.0, 'f1-score': 0}, 3: {'precision': 0.25903614457831325, 'recall': 0.08989547038327526, 'f1-score': 0.13347128815312984}, 4: {'precision': 0.0, 'recall': 0.0, 'f1-score': 0}} [repeated 3x across cluster]
(DefaultActor pid=21643) Client-side evaluation loss 0.006142548446487487 / accuracy 0.8479001135073779 [repeated 3x across cluster]
(DefaultActor pid=21643) {0: {'precision': 0.8749342891278376, 'recall': 0.8171755043742189, 'f1-score': 0.8450691222968452}, 1: {'precision': 0.8167388167388168, 'recall': 0.8118276603960942, 'f1-score': 0.8142758334486099}, 2: {'precision': 0.0, 'recall': 0.0, 'f1-score': 0}, 3: {'precision': 0.25047619047619046, 'recall': 0.04249474874777832, 'f1-score': 0.07266196988534328}, 4: {'precision': 0, 'recall': 0.0, 'f1-score': 0}} [repeated 2x across 

DEBUG flwr 2023-12-09 16:21:14,773 | server.py:187 | evaluate_round 5 received 5 results and 0 failures
DEBUG:flwr:evaluate_round 5 received 5 results and 0 failures
INFO flwr 2023-12-09 16:21:14,780 | server.py:153 | FL finished in 251.97685221399934
INFO:flwr:FL finished in 251.97685221399934
INFO flwr 2023-12-09 16:21:14,785 | app.py:226 | app_fit: losses_distributed [(1, 0.009591201297967476), (2, 0.007217007365723428), (3, 0.006256769237535114), (4, 0.005358124017170198), (5, 0.00521905913886517)]
INFO:flwr:app_fit: losses_distributed [(1, 0.009591201297967476), (2, 0.007217007365723428), (3, 0.006256769237535114), (4, 0.005358124017170198), (5, 0.00521905913886517)]
INFO flwr 2023-12-09 16:21:14,794 | app.py:227 | app_fit: metrics_distributed_fit {}
INFO:flwr:app_fit: metrics_distributed_fit {}
INFO flwr 2023-12-09 16:21:14,798 | app.py:228 | app_fit: metrics_distributed {}
INFO:flwr:app_fit: metrics_distributed {}
INFO flwr 2023-12-09 16:21:14,808 | app.py:229 | app_fit: losses_

History (loss, distributed):
	round 1: 0.009591201297967476
	round 2: 0.007217007365723428
	round 3: 0.006256769237535114
	round 4: 0.005358124017170198
	round 5: 0.00521905913886517

# Test Code

In [ ]:
#đoạn code để kiểm tra chương trình chạy đúng không
trainloader = trainloaders[0]
valloader = valloaders[0]
net = Net(param).to(DEVICE)
#chạy thử với 5 lần train
for epoch in range(2):
    train(net, trainloader, 1)
    loss, accuracy = test(net, valloader)
    print(f"Epoch {epoch+1}: validation loss {loss}, accuracy {accuracy}")

loss, accuracy = test(net, valloader)
print(f"Final test set performance:\n\tloss {loss}\n\taccuracy {accuracy}")

{0: {'precision': 0, 'recall': 0.0, 'f1-score': 0}, 1: {'precision': 0.9992776413322516, 'recall': 0.9995664595306647, 'f1-score': 0.9994220295654107}, 2: {'precision': 0.037037037037037035, 'recall': 0.012658227848101266, 'f1-score': 0.018867924528301886}, 3: {'precision': 0.998989286436224, 'recall': 0.9803610394762944, 'f1-score': 0.9895875050060071}, 4: {'precision': 0.0, 'recall': 0.0, 'f1-score': 0}}
Epoch 1: validation loss 0.00023720372456517995, accuracy 0.9989048596851472
{0: {'precision': 0, 'recall': 0.0, 'f1-score': 0}, 1: {'precision': 0.999144282808975, 'recall': 0.9995886283535128, 'f1-score': 0.9993664061892111}, 2: {'precision': 0.037037037037037035, 'recall': 0.011235955056179775, 'f1-score': 0.017241379310344827}, 3: {'precision': 0.9993935718617344, 'recall': 0.9780415430267062, 'f1-score': 0.9886022795440912}, 4: {'precision': 0.0, 'recall': 0.0, 'f1-score': 0}}
Epoch 2: validation loss 0.00043522704834746394, accuracy 0.998799557731796
{0: {'precision': 0, 'recal